In [12]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os
import shutil
from skimage import measure
from duckietown_utils import load_map, load_camera_intrinsics, load_homography, rectify, rgb_from_ros
from duckietown_utils.image_jpg_create import d8_compressed_image_from_cv_image
from obst_avoid.detector import Detector
from obst_avoid.visualizer import Visualizer
from numpy.linalg import inv

import rospy
from sensor_msgs.msg import CompressedImage
from geometry_msgs.msg import PoseArray
from visualization_msgs.msg import MarkerArray
import time
#this script is here to evaulate the perofrmance of our code on whole large datasets
robot_name='dori' #TO BE SET!!!

rospy.init_node('so_what')
detector = Detector(robot_name=robot_name)
intrinsics = load_camera_intrinsics(robot_name)
visualizer = Visualizer(robot_name=robot_name)
H = load_homography(robot_name)

obst_list = PoseArray()
marker_list = MarkerArray()


#----------------SPECIFY FILES TO READ IN ---------------------
#ASSUMES: pictures stored in folder structure ..../3/5_pics/"here_are_all_pics"
start_file=5 #represent 1_pics e.g. start_file=end_file=1 only evaluates first folder!!!
end_file=5
name_new_folder="edited"
general_path = '/home/niggi/savior_bags/3/'
dir_path = general_path+str(start_file)+"_pics"+"/"+name_new_folder
im_path = general_path+str(start_file)+"_pics"
#print dir_path

#CREATE NEW DIRECTORY
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
os.makedirs(dir_path)
#cv2.imwrite(dir+ '/' + str(i) + '.jpg',im1)

nummer=1

while(True):
    filename = im_path+ '/' + str(nummer) + '.jpg'
    im1 = cv2.imread(filename) #reads BGR
    if (im1 is None):
        #zum nächsten Ordner gehen!!!
        if (start_file>end_file):
            break
        else:
            start_file+=1
            if (start_file>end_file):
                break
            nummer=1
            dir_path = general_path+str(start_file)+"_pics"+"/"+name_new_folder
            im_path = general_path+str(start_file)+"_pics"
            #CREATE NEW DIRECTORY
            if os.path.exists(dir_path):
                shutil.rmtree(dir_path)
            os.makedirs(dir_path)
    
    
    else: #START MODIFYING THE IMAGE!!!
        #-------------HERE GOES THE REAL CODE-----------------------------------------------------------
        #-----------------------------------------------------------------------------------------------

        obst_list = detector.process_image(rectify(im1[:,:,::-1],intrinsics))
        obst_image = CompressedImage()
        obst_image.format = "jpeg"
        obst_image.data = visualizer.visualize_image(rectify(im1[:,:,::-1],intrinsics),obst_list)
        #here i want to display cropped image
        image=rgb_from_ros(obst_image.data)
        #THIS part only to visualize the cropped version -> somehow a little inefficient but keeps
        #the visualizer.py modular!!!
        #plt.imshow(image[detector.crop:,:,:]);plt.show() #the cropped image
        #plt.imshow(image);plt.show()                     #normal sized image
        #SAVE THE IMAGE
        conv = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(dir_path+ '/' + str(nummer) + '.jpg', conv)
        nummer+=1
        

print "FERTIG"

66.7431192661
66.7431192661
127
37
59
255
87
77
127
20
67
127
9
30
127
37
59
255
88
78
127
20
66
127
9
30
127
37
59
127
19
66
255
80
74
127
9
30
127
38
60
127
20
67
255
81
64
127
9
31
127
37
59
255
83
65
127
19
67
127
13
49
127
36
60
255
85
77
127
19
67
127
13
48
127
37
59
127
19
67
255
78
62
127
13
48
127
37
60
127
19
65
255
75
61
127
9
31
127
37
60
255
84
64
127
19
66
127
13
49
127
37
60
127
19
66
255
79
59
255
14
18
127
13
49
127
46
71
127
20
54
127
15
48
255
29
22
127
57
82
127
9
19
127
25
7
127
21
33
DANGEROUS OBSTACLE:
[ 0.33571094 -0.21732032]
127
22
5
127
82
96
255
16
8
127
67
64
127
43
97
127
33
118
127
10
27
127
40
59
127
6
18
127
52
89
127
103
123
127
25
59
127
19
23
255
14
20
127
55
58
DANGEROUS OBSTACLE:
[ 0.44852746 -0.37763852]
127
22
61
127
85
51
DANGEROUS OBSTACLE:
[ 0.43071431 -0.716088  ]
255
25
20
127
24
11
DANGEROUS OBSTACLE:
[ 0.22289445 -0.285604  ]
127
16
43
127
12
25
127
21
53
127
26
24
127
21
54
127
15
37
127
13
39
127
14
45
127
31
25
127
20
62
127
23
59
127
4

In [ ]:
conv = cv2.cvtColor(funky_rgb, cv2.COLOR_RGB2BGR)
cv2.imwrite( 'data/funky_rgb.png', conv)